In [ ]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import hashlib
import time
import json

# Define the Block class
class Block:
    def __init__(self, index, previous_hash, timestamp, data, nonce=0):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.nonce = nonce
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        data_str = f"{self.index}{self.previous_hash}{self.timestamp}{self.data}{self.nonce}"
        return hashlib.sha256(data_str.encode()).hexdigest()

# Define the Blockchain class
class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]

    def create_genesis_block(self):
        return Block(0, "0", int(time.time()), "Genesis Block")

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, new_block):
        new_block.previous_hash = self.get_latest_block().hash
        new_block.timestamp = int(time.time())
        new_block.hash = new_block.calculate_hash()
        self.chain.append(new_block)

# Read data from the CSV file
def read_data_from_csv(file_name):
    df = pd.read_csv(file_name)
    return df.to_dict(orient="records")

# Create a transaction dictionary
def create_transaction(row):
    transaction = {
        "TransactionNo": row["TransactionNo"],
        "Date": row["Date"],
        "ProductNo": row["ProductNo"],
        "ProductName": row["ProductName"],
        "Price": row["Price"],
        "Quantity": row["Quantity"],
        "CustomerNo": row["CustomerNo"],
        "Country": row["Country"],
    }
    return transaction

# Create the blockchain
shop_chain = Blockchain()
data = read_data_from_csv("shop.data.csv")

# Create blocks for each transaction
for row in data:
    transaction = create_transaction(row)
    new_block = Block(len(shop_chain.chain), shop_chain.get_latest_block().hash, int(time.time()), json.dumps(transaction))
    shop_chain.add_block(new_block)

# Create a tkinter GUI
app = tk.Tk()
app.title("Blockchain Transaction Viewer")
app.geometry("600x800")  # Adjusted the window size
app.configure(bg="#0033cc")

style = ttk.Style()
style.configure("TLabel", font=("Arial", 14), padding=5)
style.configure("TButton", font=("Arial", 14), bg="red", fg="black", padding=5)

# Function to display transactions
def show_transactions():
    transactions = []
    for block in shop_chain.chain:
        try:
            block_data = json.loads(block.data)
            transaction_no = block_data["TransactionNo"]
            transactions.append(f"Block No: {block.index}, Transaction No: {transaction_no}")
        except json.JSONDecodeError:
            pass
    transactions_text.set("\n".join(transactions))

# Function to find and display transaction details
def find_transaction():
    transaction_no = entry.get()
    result_text.set("")  # Clear previous result
    for block in shop_chain.chain:
        try:
            block_data = json.loads(block.data)
            if str(block_data["TransactionNo"]) == transaction_no:
                formatted_data = "\n".join([f"{key}: {value}" for key, value in block_data.items()])
                result_text.set(f"Block No: {block.index}\nTransaction Details:\n{formatted_data}\nTimestamp: {block.timestamp}\nHash: {block.hash}")
                return
        except json.JSONDecodeError:
            pass
    result_text.set("Transaction not found!")

show_button = ttk.Button(app, text="Show Transactions", command=show_transactions)
show_button.pack(pady=10)

transactions_text = tk.StringVar()
transactions_label = ttk.Label(app, textvariable=transactions_text, wraplength=580)
transactions_label.pack()

label = tk.Label(app, text="Enter Transaction Number:")
label.pack(pady=10)

entry = ttk.Entry(app)
entry.pack()

find_button = tk.Button(app, text="Find Transaction", command=find_transaction, bg="hotpink", font=("Arial", 14), padx=5, pady=5, fg="white")
find_button.pack(pady=10)

result_text = tk.StringVar()
result_label = ttk.Label(app, textvariable=result_text, wraplength=580)
result_label.pack()

app.mainloop()